In [1]:
import spacy

In [2]:
nlp = spacy.load("pt_core_news_lg")

In [3]:
import pandas as pd

Experimentos de identificação e extração de aspectos

In [4]:
df = pd.read_csv("treino_shared_task.csv", delimiter=";")
df2 = pd.read_csv("test_task2.csv", delimiter=";")

In [5]:
ontology_df1 = pd.read_csv("aspects_hontology - single_words.csv",header=None)
ontology_df2 = pd.read_csv("aspects_hontology - multi_words.csv", header=None)

copy_ontology_df1 = ontology_df1.copy()
copy_ontology_df2 = ontology_df2.copy()

In [6]:
import unidecode
import re
def remove_punctuation(text):
    text = unidecode.unidecode(text)
    re_special_characters = r"\| | \\ | \/ | \( | \) | \{ | \} | \[ | \] |\-"
    text = re.sub(re_special_characters, " ", text)
    text = text.replace(r',',' ')
    text = text.replace(r'!',' ')
    text = text.replace(r'?',' ')
    text = text.replace(r'.',' ')
    text = text.replace(r'(',' ')
    text = text.replace(r')',' ')
    text = text.replace(r':',' ')
    text = text.replace(r';',' ')
    text = text.replace(r'$',' ')
    text = text.replace(r'*',' ')
    text = text.replace(r'"',' ')
    
    return text

In [7]:
def lower_case(text):
    return text.lower()

In [8]:
df2['review'] = df2['review'].apply(remove_punctuation)
df2['review'] = df2['review'].apply(lower_case)

In [9]:
ontology_df1[0] = ontology_df1[0].apply(remove_punctuation)
ontology_df2[0] = ontology_df2[0].apply(remove_punctuation)

In [10]:
ontoly_single_words = ontology_df1.iloc[:,0].tolist()
ontology_multi_words = ontology_df2.iloc[:,0].tolist()

In [11]:
def create_quadruplet(A,B,C,D): 
   return [A, B, C, D]

In [12]:
df['review'] = df['review'].apply(lower_case)

In [13]:
todos_aspectos = []
aspectos_train = df['aspect'].unique()
aspectos_test = df2['aspect'].unique()

In [15]:
todos_aspectos.extend(aspectos_train)
todos_aspectos.extend(aspectos_test)
todos_aspectos.append("shoppings")
todos_aspectos.append("tomadas")

todos_aspectos = list(set(todos_aspectos))
readable_aspects = todos_aspectos.copy()

todos_aspectos = [remove_punctuation(x) for x in todos_aspectos]

In [16]:
def extract_single_word_aspects(dataframe):
    #dataframe['review'] = dataframe['review'].apply(remove_punctuation)
    lista_review = get_tags(dataframe)
    aspect_list = []
    for item in lista_review:
        aspects = []
        for i in item:
            if (i[3] in todos_aspectos):
                if i[3] == "shoppings":
                    aspects.append("shopping")
                else:
                    aspects.append(i[3])
            elif (i[0] in todos_aspectos):
                if i[0] == "tomadas":
                    aspects.append("tomada")
                else:
                    aspects.append(i[0])
            elif i[2] in ['ROOT', 'nsubj', 'obl'] and i[1] == 'NOUN':
                if exists_in_ontology(i[0]):
                    aspects.append(i[0])
                elif exists_in_ontology(i[3]):
                    aspects.append(i[3])
        aspect_list.append(list(set(aspects)))
    return aspect_list

def get_tags(dataframe):
    lista_review =[]
    for i in range(dataframe['review'].shape[0]):
        review = []
        doc = nlp(dataframe['review'].iloc[i])
        for token in doc:
            quadrupla = create_quadruplet(token.text, token.pos_, token.dep_, token.lemma_)
            review.append(quadrupla)
        lista_review.append(review)
    return lista_review

def exists_in_ontology(word):
    word = remove_punctuation(word)
    return word in ontoly_single_words


def extract_multi_word_aspects(review):
    #review = lower_case(review)
    #review = remove_punctuation(review)
    
    return list({word for word in ontology_multi_words if re.search(word, review)})
    

In [28]:
df_test = pd.read_csv("train_aspects.csv")
df_test['review'] = df_test['review'].apply(remove_punctuation)
df_test['review'] = df_test['review'].apply(lower_case)

In [29]:
df_test

,review,aspect
0,o hotel e perto de todos os pontos principais ...,"['quarto', 'preço', 'café da manhã', 'hotel']"
1,viajamos eu e minha irma o hotel tem uma extr...,"['elevador', 'café da manhã', 'hotel']"
2,estive por 8 dias hospedado neste hotel com mi...,"['café da manhã', 'recepção', 'hotel']"
3,adorei a estadia porto alegre foi sensacional...,"['hotel', 'cidade', 'turista', 'funcionários',..."
4,o hotel tem otima localizacao fizemos varios ...,"['internet', 'quarto', 'hotel', 'recepção', 'l..."
...,...,...
842,nao conheco outros hoteis em ny mas conheco e...,['carpete']
843,os quartos sao excelentes tudo novo limpo e ...,['hotel']
844,o hotel fica no chelsea perto da macys do ma...,['localização']
845,ficamos hospedados por 8 dias no hotel a equ...,['hotel']


In [31]:
reviews = df_test['review'].tolist()

In [32]:
df_test['annotated_aspects'] = df_test['aspect']

In [33]:
df_test.head()

,review,aspect,annotated_aspects
0,o hotel e perto de todos os pontos principais ...,"['quarto', 'preço', 'café da manhã', 'hotel']","['quarto', 'preço', 'café da manhã', 'hotel']"
1,viajamos eu e minha irma o hotel tem uma extr...,"['elevador', 'café da manhã', 'hotel']","['elevador', 'café da manhã', 'hotel']"
2,estive por 8 dias hospedado neste hotel com mi...,"['café da manhã', 'recepção', 'hotel']","['café da manhã', 'recepção', 'hotel']"
3,adorei a estadia porto alegre foi sensacional...,"['hotel', 'cidade', 'turista', 'funcionários',...","['hotel', 'cidade', 'turista', 'funcionários',..."
4,o hotel tem otima localizacao fizemos varios ...,"['internet', 'quarto', 'hotel', 'recepção', 'l...","['internet', 'quarto', 'hotel', 'recepção', 'l..."


In [34]:
df_test['extracted_SW_aspects'] = extract_single_word_aspects(df_test)
df_test['extracted_MW_aspects'] = df_test['review'].apply(extract_multi_word_aspects)

In [35]:
df_test['complete_extracted_aspects'] = df_test['extracted_SW_aspects'] + df_test['extracted_MW_aspects']

In [36]:
df_test

,review,aspect,annotated_aspects,extracted_SW_aspects,extracted_MW_aspects,complete_extracted_aspects
0,o hotel e perto de todos os pontos principais ...,"['quarto', 'preço', 'café da manhã', 'hotel']","['quarto', 'preço', 'café da manhã', 'hotel']","[preco, quarto, hotel]",[cafe da manha],"[preco, quarto, hotel, cafe da manha]"
1,viajamos eu e minha irma o hotel tem uma extr...,"['elevador', 'café da manhã', 'hotel']","['elevador', 'café da manhã', 'hotel']","[elevador, cama, quarto, hotel]",[cafe da manha],"[elevador, cama, quarto, hotel, cafe da manha]"
2,estive por 8 dias hospedado neste hotel com mi...,"['café da manhã', 'recepção', 'hotel']","['café da manhã', 'recepção', 'hotel']","[familia, recepcao, hotel]",[cafe da manha],"[familia, recepcao, hotel, cafe da manha]"
3,adorei a estadia porto alegre foi sensacional...,"['hotel', 'cidade', 'turista', 'funcionários',...","['hotel', 'cidade', 'turista', 'funcionários',...","[elevador, localizacao, hotel, funcionarios, c...",[centro comercial],"[elevador, localizacao, hotel, funcionarios, c..."
4,o hotel tem otima localizacao fizemos varios ...,"['internet', 'quarto', 'hotel', 'recepção', 'l...","['internet', 'quarto', 'hotel', 'recepção', 'l...","[internet, recepcao, localizacao, quarto, hotel]",[cafe da manha],"[internet, recepcao, localizacao, quarto, hote..."
...,...,...,...,...,...,...
842,nao conheco outros hoteis em ny mas conheco e...,['carpete'],['carpete'],"[carpete, quarto, hotel]",[],"[carpete, quarto, hotel]"
843,os quartos sao excelentes tudo novo limpo e ...,['hotel'],['hotel'],"[quarto, hotel]",[],"[quarto, hotel]"
844,o hotel fica no chelsea perto da macys do ma...,['localização'],['localização'],"[lojas, farmacia, localizacao, hotel]",[],"[lojas, farmacia, localizacao, hotel]"
845,ficamos hospedados por 8 dias no hotel a equ...,['hotel'],['hotel'],"[recepcao, internet, lavanderia, quarto, hotel]",[],"[recepcao, internet, lavanderia, quarto, hotel]"


In [38]:
import ast
def found_all_aspects(lista1, lista2):
    auxlist = ast.literal_eval(lista1)
    auxlist = list(set(auxlist))
    auxlist = sorted(auxlist)
    auxlist2 = sorted(lista2)


    common_elements = list(set(auxlist).intersection(set(auxlist2)))
    return "YES" if sorted(common_elements) == auxlist else "NO"

In [39]:
def missing_aspects(lista1, lista2):
    auxlist = ast.literal_eval(lista1)
    auxlist = [remove_punctuation(i) for i in auxlist]
    auxlist2 = [remove_punctuation(i) for i in lista2]
    return sorted(list(set(auxlist).difference(set(auxlist2))))

def extra_aspects(lista1, lista2):
    lista2 = ast.literal_eval(lista2)
    set_difference = set(lista1) - set(lista2)
    list_difference = list(set_difference)
    return sorted(list_difference)


In [40]:
def get_readable_aspects(lista):
    s = pd.Series(readable_aspects)
    copy_ontology_df1['no_punctuation'] = copy_ontology_df1[0].apply(remove_punctuation)
    copy_ontology_df2['no_punctuation'] = copy_ontology_df2[0].apply(remove_punctuation)
    frame = pd.concat([copy_ontology_df1, copy_ontology_df2, s])
    frame['no_punctuation'] = frame[0].apply(remove_punctuation)
    frame.drop_duplicates(subset=['no_punctuation'], keep='first', inplace=True)


    auxlist = [frame[frame['no_punctuation'] == i][0] for i in lista]
    return sorted(flatten(auxlist))

def flatten(l):
    return [item for sublist in l for item in sublist]

In [41]:
df_test['redable_aspects'] = df_test['complete_extracted_aspects'].apply(get_readable_aspects)

In [42]:
df_test

,review,aspect,annotated_aspects,extracted_SW_aspects,extracted_MW_aspects,complete_extracted_aspects,redable_aspects
0,o hotel e perto de todos os pontos principais ...,"['quarto', 'preço', 'café da manhã', 'hotel']","['quarto', 'preço', 'café da manhã', 'hotel']","[preco, quarto, hotel]",[cafe da manha],"[preco, quarto, hotel, cafe da manha]","[café da manhã, hotel, preço, quarto]"
1,viajamos eu e minha irma o hotel tem uma extr...,"['elevador', 'café da manhã', 'hotel']","['elevador', 'café da manhã', 'hotel']","[elevador, cama, quarto, hotel]",[cafe da manha],"[elevador, cama, quarto, hotel, cafe da manha]","[café da manhã, cama, elevador, hotel, quarto]"
2,estive por 8 dias hospedado neste hotel com mi...,"['café da manhã', 'recepção', 'hotel']","['café da manhã', 'recepção', 'hotel']","[familia, recepcao, hotel]",[cafe da manha],"[familia, recepcao, hotel, cafe da manha]","[café da manhã, família, hotel, recepção]"
3,adorei a estadia porto alegre foi sensacional...,"['hotel', 'cidade', 'turista', 'funcionários',...","['hotel', 'cidade', 'turista', 'funcionários',...","[elevador, localizacao, hotel, funcionarios, c...",[centro comercial],"[elevador, localizacao, hotel, funcionarios, c...","[centro comercial, cidade, elevador, funcionár..."
4,o hotel tem otima localizacao fizemos varios ...,"['internet', 'quarto', 'hotel', 'recepção', 'l...","['internet', 'quarto', 'hotel', 'recepção', 'l...","[internet, recepcao, localizacao, quarto, hotel]",[cafe da manha],"[internet, recepcao, localizacao, quarto, hote...","[café da manhã, hotel, internet, localização, ..."
...,...,...,...,...,...,...,...
842,nao conheco outros hoteis em ny mas conheco e...,['carpete'],['carpete'],"[carpete, quarto, hotel]",[],"[carpete, quarto, hotel]","[carpete, hotel, quarto]"
843,os quartos sao excelentes tudo novo limpo e ...,['hotel'],['hotel'],"[quarto, hotel]",[],"[quarto, hotel]","[hotel, quarto]"
844,o hotel fica no chelsea perto da macys do ma...,['localização'],['localização'],"[lojas, farmacia, localizacao, hotel]",[],"[lojas, farmacia, localizacao, hotel]","[farmácia, hotel, localização, lojas]"
845,ficamos hospedados por 8 dias no hotel a equ...,['hotel'],['hotel'],"[recepcao, internet, lavanderia, quarto, hotel]",[],"[recepcao, internet, lavanderia, quarto, hotel]","[hotel, internet, lavanderia, quarto, recepção]"


In [48]:
#df_test['found_all_aspects'] = df2_copy.apply(lambda row: found_all_aspects(row['annotated_aspects'], row['redable_aspects']), axis=1)

In [49]:
#df_test['extra_aspects'] = df2_copy.apply(lambda row: extra_aspects(row['redable_aspects'], row['annotated_aspects']), axis=1)

In [50]:
#df_test['missing_aspects'] = df2_copy.apply(lambda row: missing_aspects(row['annotated_aspects'], row['complete_extracted_aspects']), axis=1)

In [ ]:
#df_test['found_all_aspects'].value_counts()

In [569]:
#df_test.to_csv("extracted_aspects_train_dataset.csv", header=True, index=False)

In [46]:
df_final = pd.read_csv("test_task1.csv", delimiter=';')

In [47]:
df_final['review'] = df_final['review'].apply(remove_punctuation)
df_final['review'] = df_final['review'].apply(lower_case)

In [48]:
df_final['extracted_SW_aspects'] = extract_single_word_aspects(df_final)
df_final['extracted_MW_aspects'] = df_final['review'].apply(extract_multi_word_aspects)

df_final['complete_extracted_aspects'] = df_final['extracted_SW_aspects'] + df_final['extracted_MW_aspects']

In [49]:
df_final['redable_aspects'] = df_final['complete_extracted_aspects'].apply(get_readable_aspects)

In [50]:
df_final.drop(df_final.columns.difference(['review','redable_aspects','id']), 1, inplace=True)

In [51]:
df_final

,id,review,redable_aspects
0,0,a 10 metros do hotel tem um pub legal e em se...,"[café da manhã, elevador, hotel, localização, ..."
1,1,a imensa falha e a falta de wifi pelo menos n...,"[aeroporto, custo-benefício, hotel, limpeza, r..."
2,2,a limpeza deixa um pouco a desejar por 2 veze...,"[limpeza, minibar, secador, toalha]"
3,3,a localizacao do hotel e privilegiada contudo...,"[apartamento, café da manhã, hotel, localização]"
4,4,a localizacao do hotel e otima entre a 5ae 6a...,"[hotel, localização, quarto, rua]"
...,...,...,...
252,252,otima localizacao para ficar andando pela time...,"[atendimento, café da manhã, cama, casal, frig..."
253,253,otima localizacao custo beneficio excelente ...,"[aeroporto, café da manhã, custo-benefício, el..."
254,254,otima localizacao predio moderno com todas as...,"[conforto, funcionários, hotel, localização, s..."
255,255,otima localizacao proximo a uma estacao de me...,"[café da manhã, elevador, funcionários, hotel,..."


In [ ]:
#df_final.to_csv("extracted_aspects_final_result.csv", header=True, index=False)